# Build the 2D reservoir surface and the 3D VTI model

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

from IPython.display import HTML

import seppy
import os

rcParams['font.size'] = 8
rcParams['font.family'] = 'sans-serif'

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# utility functions

def getXYZ(md, traj,md0=-1):
    """Compute x,y,z coordinates for a given measured depth"""

    n = traj.shape[1]
    
    if md0<0:
        l=traj[2,0]
    else:
        l=md0
    i=1
    while ((l<md) and (i<n)):
        l += math.sqrt((traj[2,i]-traj[2,i-1])**2+(traj[1,i]-traj[1,i-1])**2+(traj[0,i]-traj[0,i-1])**2)
        i += 1
        
    if (i==n):
        print("Warning: the end of the trajectory has been reached")
        x=traj[0,n-1]
        y=traj[1,n-1]
        z=traj[2,n-1]
    else:
        imin = i - 2
        imax=imin+1
        if (imin<0):
            imin=0
            imax=0
        lmin=l-math.sqrt((traj[2,imax]-traj[2,imax-1])**2+(traj[1,imax]-traj[1,imax-1])**2+(traj[0,imax]-traj[0,imax-1])**2)
        wmax=(md-lmin)/(l-lmin)
        wmin=1-wmax
        x=wmin*traj[0,imin]+wmax*traj[0,imax]
        y=wmin*traj[1,imin]+wmax*traj[1,imax]
        z=wmin*traj[2,imin]+wmax*traj[2,imax]
    
    return x,y,z

def ChannelToDepth(Channel):
    """Compute DAS channel measured depth from channel number (between 1 and 3776)
    The conversion function is provided by Chevron"""
    
    FibreLengthMultiplier = 1.02095238387
    ZeroOffset_m          =-2.41333217886
    SpatialResolution_m   = 1.0
    DasFibreStretch       = 1.0056

    Depth_in_meters       = ( (float(Channel) 
                             * SpatialResolution_m 
                             * FibreLengthMultiplier )
                             + ZeroOffset_m) / DasFibreStretch

    return Depth_in_meters

def rotationY(angle):
    """Build rotation matrix around y-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,0,-s],[0,1,0],[s,0,c]))

    return R

def rotationZ(angle):
    """Build rotation matrix around z-axis ; the angle is in degrees"""

    c = math.cos(math.radians(angle))
    s = math.sin(math.radians(angle))
    R=np.array(([c,-s,0],[s,c,0],[0,0,1]))

    return R

def rotateYZ(xyz,theta,phi):
    """Rotate coordinates around y- and z-axes ; angles are in degrees"""

    Ry = rotationY(theta)
    Rz = rotationZ(phi)
    v = np.matmul(Rz,xyz)
    v = np.matmul(Ry,v)

    return v

def findYZ(x,traj):
    """Find y and z given x along a trajectory"""
    
    n=traj.shape[1]
    answer=False
    i=0
    while i<n and answer==False:
        xi=traj[0,i]
        if x>xi:
            i+=1
        else:
            answer=True
    if i==n:
        print("WARNING: end of trajectory reached")
        y = traj[1,n-1]
        z = traj[2,n-1]
    else:
        xi0=traj[0,i-1]
        wi = (x-xi0)/(xi-xi0)
        wi0=1.-wi
        y = wi0*traj[1,i-1]+wi*traj[1,i]
        z = wi0*traj[2,i-1]+wi*traj[2,i]
    
    return np.array([y,z]) 

### Build the bottom reservoir surface

In [ ]:
# read the formation bottom horizon curve (rotated)

sep = seppy.sep()

axes, data = sep.read_file(datapath+"../../ch4/dat/ch4_hrz.H")
hrz1d = data.reshape(axes.n,order='F').T

# convert from km to m
ox_hrz=axes.o[0]*1000
dx_hrz=axes.d[0]*1000
hrz1d *= 1000


# Read the full deviated well (monitor) trajectory
# The file contains 4 columns: Measured Depth (ft), Inclination (degrees), NS (ft), EW (ft)

inputfile="../../../input_data/auxiliary_data/MonitorWell_WellSurvey.csv"

df=pd.read_csv(inputfile, sep=',',header=0)
traj=df.values[:,0:4]

# convert the MD to TVD and convert (ft) to (m)
n=traj.shape[0]
z = np.zeros(n)

x = traj[:,3]*0.3048
y = traj[:,2]*0.3048

z[0] = 0
for i in range(1, n):
    z[i] = z[i-1] + (traj[i,0]-traj[i-1,0])*math.cos(math.radians(traj[i,1]))

z = z*0.3048

# save the trajectory
trajectory = np.array([x,y,z])

In [ ]:
# Read "trajectories" of monitor, Western, and Eastern wells ; the trajectories are actually the nominal perforation locations

monitor=pd.read_excel("../../../input_data/auxiliary_data/PerforationInfo4SEP-v1.xlsx", sheet_name='Monitor Well')
east=pd.read_excel("../../../input_data/auxiliary_data/PerforationInfo4SEP-v1.xlsx", sheet_name='East Treat Well')
west=pd.read_excel("../../../input_data/auxiliary_data/PerforationInfo4SEP-v1.xlsx", sheet_name='West Treat Well')

# Calibrate the trajectories to the reference monitor one
monitor_traj=np.array([monitor["DX-Monitor"],monitor["DY-Monitor"],monitor["Z"]])
east_traj=np.array([east["DX-Monitor"],east["DY-Monitor"],east["Z"]])
west_traj=np.array([west["DX-Monitor"],west["DY-Monitor"],west["Z"]])

monitor_traj=np.flip(monitor_traj,axis=1)*0.3048 # flip the order and convert from ft to meters
east_traj=np.flip(east_traj,axis=1)*0.3048
west_traj=np.flip(west_traj,axis=1)*0.3048

monitor_traj[2,:] *= -1 # from negative to positive depth
east_traj[2,:] *= -1 
west_traj[2,:] *= -1

# calibration values estimated visually by comparison with reference (full) monitor trajectory
x_calib = 4
y_calib = 62
z_calib = 387.2

monitor_traj[0,:] += x_calib
east_traj[0,:] += x_calib 
west_traj[0,:] += x_calib
monitor_traj[1,:] += y_calib
east_traj[1,:] += y_calib 
west_traj[1,:] += y_calib
monitor_traj[2,:] += z_calib
east_traj[2,:] += z_calib 
west_traj[2,:] += z_calib

# rotate the coordinates
theta = -0.648114658
phi = 54.4

trajectory_rot = np.copy(trajectory)
monitor_traj_rot = np.copy(monitor_traj)
east_traj_rot = np.copy(east_traj)
west_traj_rot = np.copy(west_traj)

for i in range(trajectory.shape[1]):
    trajectory_rot[:,i] = rotateYZ(trajectory[:,i],theta,phi)
    
for i in range(monitor_traj.shape[1]):
    monitor_traj_rot[:,i] = rotateYZ(monitor_traj[:,i],theta,phi)
    
for i in range(east_traj.shape[1]):
    east_traj_rot[:,i] = rotateYZ(east_traj[:,i],theta,phi)
    
for i in range(west_traj.shape[1]):
    west_traj_rot[:,i] = rotateYZ(west_traj[:,i],theta,phi)

In [ ]:
nhrz=hrz1d.shape[0]
i0=68

plt.figure(figsize=(12,6),dpi=300)
ax = plt.axes(projection='3d')
ax.plot3D(0.001*east_traj[0,:],0.001*east_traj[1,:],0.001*east_traj[2,:],c='c',Linestyle='--',label="NE well")
# ax.plot3D(0.001*trajectory[0,i0:],0.001*trajectory[1,i0:],0.001*trajectory[2,i0:],c='b',label="DAS well")
ax.plot3D(0.001*monitor_traj[0,:],0.001*monitor_traj[1,:],0.001*monitor_traj[2,:],c='r',label="DAS well")
ax.plot3D(0.001*west_traj[0,:],0.001*west_traj[1,:],0.001*west_traj[2,:],c='g',label="SW well")
# ax.plot3D(np.linspace(0.001*ox_hrz,0.001*ox_hrz+0.001*(nhrz-1)*dx_hrz,nhrz),np.zeros((nhrz,1))+0.001*np.mean(trajectory_rot[1,i0:]),0.001*hrz1d[:],c='y',label="Hrz")
plt.gca().invert_zaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend()
# plt.gca().set_zlim(1980,1940)
ax.set_xlabel("Easting (km)")
ax.set_ylabel("Northing (km)")
ax.set_zlabel("Depth (km)")
ax.locator_params(nbins=5, axis='y')
ax.view_init(10, -80)
plt.savefig(figpath+'ch6_wells_3d.png',bbox_inches='tight',format='png')

In [ ]:
plt.figure(figsize=(6.66,6),dpi=300)
plt.plot(0.001*east_traj[0,:],0.001*east_traj[1,:],c='c',Linestyle='--',label="NE well")
plt.plot(0.001*monitor_traj[0,:],0.001*monitor_traj[1,:],c='r',label="DAS well")
plt.plot(0.001*west_traj[0,:],0.001*west_traj[1,:],c='g',label="SW well")
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.xlabel("Easting (km)")
plt.ylabel("Northing (km)")
plt.tick_params(axis='both', which='major', labelsize=8)
plt.savefig(figpath+'ch6_wells_map.png',bbox_inches='tight',format='png')

In [ ]:
# smooth the trajectories needed to estimate the formation curvature in the y-direction
npts=3
order=3
east_traj_rot_smth = signal.savgol_filter(east_traj_rot,int(east_traj_rot.shape[1]/npts),order,axis=1)
west_traj_rot_smth = signal.savgol_filter(west_traj_rot,int(west_traj_rot.shape[1]/npts),order,axis=1)
trajectory_rot_smth = signal.savgol_filter(trajectory_rot[:,i0:],1+int(trajectory_rot[:,i0:].shape[1]/npts),order,axis=1)

# Extend the horizon to a surface in the y-direction by fitting polynomials using the smoothed well trajectories
nwest=300//2 # size of the y-extension
neast=280//2
nx_hrz = hrz1d.shape[0]
ny_hrz = nwest+neast
hrz2d=np.zeros((nx_hrz,ny_hrz))

oy_hrz=np.min(west_traj_rot[1,:])-10 # y-origin (a bit away from the farthest point in the West well)
dy_hrz=dx_hrz
y_all=np.linspace(oy_hrz,oy_hrz+(ny_hrz-1)*dy_hrz,ny_hrz)

ix0 = int((350 - ox_hrz)/dx_hrz) # skip parts at the edges because not enough well trajectories
ixmax=hrz1d.shape[0]-87

p=np.zeros((nx_hrz,3))
yz_west=np.zeros((nx_hrz,2))
yz_east=np.zeros((nx_hrz,2))
yz_monitor=np.zeros((nx_hrz,2))
yz_monitor_orig=np.zeros((nx_hrz,2))

for ix in range(ix0,ixmax):
    x = ox_hrz + ix*dx_hrz
    yz_west[ix,:] = findYZ(x,west_traj_rot_smth)
    yz_east[ix,:] = findYZ(x,east_traj_rot_smth)    
    yz_monitor[ix,:] = findYZ(x,trajectory_rot_smth)
    yz_monitor_orig[ix,:]=findYZ(x,trajectory_rot)
  
    # polynomial fitting
    p[ix,:] = np.polyfit([yz_west[ix,0],yz_monitor[ix,0],yz_east[ix,0]],[yz_west[ix,1],yz_monitor[ix,1],yz_east[ix,1]],deg=2)
    hrz2d[ix,:] = p[ix,0]*y_all**2 + p[ix,1]*y_all + p[ix,2]
    
    # hang the 1D formation horizon at the non-smoothed monitor well location
    iy_monitor = int(round((yz_monitor_orig[ix,0]-oy_hrz)/dy_hrz))
    hrz2d[ix,:] = hrz2d[ix,:] + hrz1d[ix] - hrz2d[ix,iy_monitor]

# extrapolate at the edges
hrz2d[:ix0,:] = hrz2d[ix0,:]
hrz2d[ixmax:,:] = hrz2d[ixmax-1,:]
yz_monitor_orig[:ix0,:] = yz_monitor_orig[ix0,:]
yz_monitor_orig[ixmax:,:] = yz_monitor_orig[ixmax-1,:]

In [ ]:
ix=ixmax-50
z_fit =p[ix,0]*y_all**2 + p[ix,1]*y_all + p[ix,2]
plt.scatter([yz_west[ix,0],yz_monitor[ix,0],yz_east[ix,0]],[yz_west[ix,1],yz_monitor[ix,1],yz_east[ix,1]])
plt.plot(y_all,z_fit,LineStyle='--')
plt.xlabel("Y (m)")
plt.ylabel("Depth (m)")
plt.gca().invert_yaxis()

In [ ]:
plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')
x_all_2d = np.outer(np.linspace(ox_hrz,ox_hrz+(nhrz-1)*dx_hrz,nhrz), np.ones(ny_hrz))
y_all_2d = np.outer(y_all,np.ones(nhrz)).T

ax.plot_surface(x_all_2d, y_all_2d, hrz2d,cmap='viridis', edgecolor='none')
ax.plot3D(east_traj_rot[0,:],east_traj_rot[1,:],east_traj_rot[2,:],c='c',label="East")
ax.plot3D(trajectory_rot[0,i0:],trajectory_rot[1,i0:],trajectory_rot[2,i0:],c='b',label="Monitor reference")
ax.plot3D(west_traj_rot[0,:],west_traj_rot[1,:],west_traj_rot[2,:],c='g',label="West")
plt.gca().invert_zaxis()
plt.gca().set_zlim(1980,1940)
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")
plt.tick_params(axis='both', which='major', labelsize=8)
ax.view_init(10, -90)
plt.show()

In [ ]:
x_all_2d = np.outer(np.linspace(ox_hrz,ox_hrz+(nhrz-1)*dx_hrz,nhrz), np.ones(ny_hrz))
y_all_2d = np.outer(y_all,np.ones(nhrz)).T

plt.figure(figsize=(12,6),dpi=300)
ax = plt.axes(projection='3d')

ax.plot3D(0.001*east_traj_rot[0,:],0.001*east_traj_rot[1,:],0.001*east_traj_rot[2,:],c='c',Linestyle='--',label="NE well")
ax.plot3D(0.001*monitor_traj_rot[0,:],0.001*monitor_traj_rot[1,:],0.001*monitor_traj_rot[2,:],c='r',label="DAS well")
ax.plot3D(0.001*west_traj_rot[0,:],0.001*west_traj_rot[1,:],0.001*west_traj_rot[2,:],c='g',label="SW well")
ax.plot_surface(0.001*x_all_2d, 0.001*y_all_2d, 0.001*hrz2d,cmap='jet', edgecolor='none',alpha=0.5)
plt.gca().invert_zaxis()
plt.gca().tick_params(axis='both', which='major', labelsize=6)
plt.legend()
ax.set_xlabel("X (km)")
ax.set_ylabel("Y (km)")
ax.set_zlabel("Depth* (km)")
ax.locator_params(nbins=5, axis='y')
ax.view_init(20, -40)
plt.savefig(figpath+'ch6_wells_3d_rot.png',bbox_inches='tight',format='png')

### Build the 3D VTI model

In [ ]:
# read the 2D VTI model (it has the same spatial extension as the formation horizon)

axes, data = sep.read_file(datapath+"../../ch5/dat/ch5_model.H")
model2d = data.reshape(axes.n,order='F').T

Zn=axes.n[0]
Xn=axes.n[1]
Zo=axes.o[0]
Xo=axes.o[1]
Zd=axes.d[0]
Xd=axes.d[1]

Estimate $\gamma$ from Vs_fast horizontal (2.3 km/s) and Vs_slow horizontal (1.7 km/s): this gives $\gamma = ((\frac{V_s(90)}{V_s(0)})^2 - 1)/2 = 0.42$.\
Estimate $\epsilon$ from Vp horizontal (3.7 km/s) and Vp vertical (2.5 km/s): this gives $\epsilon = ((\frac{V_p(90)}{V_p(0)})^2 - 1)/2 = 0.60$.\
Therefore the ratio $\frac{\gamma}{\epsilon}=0.7$

In [ ]:
# initialize the 3D VTI model (Gamma is set to 0.7*Epsilon and may need calibration later)
model3d = np.zeros((6,ny_hrz,Xn,Zn))
factor=0.7 # ratio gamma/epsilon

# extract the 1D profile corresponding to the corner (x=ox) of the 2D model 
profile = model2d[:,0,:]

# Extrapolate the profile to populate the 3D model
yo = np.median(yz_monitor_orig[:,0]) # y-location (in m) to place the non shifted profile from the 2D model, I take the median of the non-smoothed monitor trajectory. The x-location is at the origin
iymod3d = int(round((yo-oy_hrz)/dy_hrz)) # index of the y-location above

for iy in range(model3d.shape[1]):
     for ix in range(model3d.shape[2]):
        shift=(hrz2d[ix,iy]-hrz2d[0,iymod3d])/(1000*Zd)
        model3d[:5,iy,ix,:] = ndimage.shift(profile,[0,shift],mode='nearest')

# Populate Gamma using Epsilon
model3d[5,:,:,:] = factor*model3d[4,:,:,:]

In [ ]:
plt.plot(profile[0,:])
plt.plot(model3d[0,iymod3d,0,:],LineStyle='--')
plt.show()

In [ ]:
# Save the 2D surface and the 3D VTI model (convert the depth surface to km)
sep.write_file(datapath+"ch6_model3d.H", np.transpose(model3d), ds=np.array([Zd,Xd,0.001*dy_hrz]), os=np.array([Zo,Xo,0.001*oy_hrz]), dpath=datapath)
sep.write_file(datapath+"ch6_hrz2d.H", 0.001*hrz2d, ds=np.array([Xd,0.001*dy_hrz]), os=np.array([Xo,0.001*oy_hrz]), dpath=datapath)